In [3]:
import joblib
import numpy as np

rf_model = joblib.load('phishing_detector.pkl')

# parameters = [33,26,0,2,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,1,0.03030303,0.038461538,0,0,0,0,0,2,0,0,1,0,1,0,2,0,8,8,0,13,13,0,10.5,10.5,0.0,0,1,0,0,0,0,10,0.1,0.9,0,0,0,0.7777777779999999,0,0.5555555560000001,0,1,0.0,0,0.0,0.0,0,0,0,33.33333333,0,0,0,1,0,0,374,7295,0,0,1,5]

parameters = np.array(parameters).reshape(1, -1)

y_pred = rf_model.predict(parameters)

print("Phishing" if y_pred[0] == 1 else "Legitimate")

Phishing


c:\Users\shaur\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
